In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import json
import random
import pandas as pd
import shutil

<IPython.core.display.Javascript object>

In [3]:
!mkdir prepare_data/obj_det_ssd_json/train_imgs
!mkdir prepare_data/obj_det_ssd_json/valid_imgs
!mkdir prepare_data/obj_det_ssd_json/train_annots
!mkdir prepare_data/obj_det_ssd_json/valid_annots

<IPython.core.display.Javascript object>

In [4]:
gt_df = pd.read_csv("prepare_data/obj_det_ssd_json/all_images_gt/od_labels.csv",)
print(gt_df.shape)
gt_df.head()

(161, 8)


,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,apple,76,31,410,350,7385219.jpg,705,724
1,orange,169,172,396,465,7385219.jpg,705,724
2,apple,143,54,357,341,3503298.jpg,730,725
3,banana,101,186,481,363,3503298.jpg,730,725
4,apple,81,40,473,332,3503298.jpg,730,725


<IPython.core.display.Javascript object>

In [5]:
train_valid_split = 0.7
nimages = gt_df["image_name"].nunique()
ntrain = int(train_valid_split * nimages)
nvalid = nimages - ntrain
print(nimages, ntrain, nvalid)

100 70 30


<IPython.core.display.Javascript object>

In [6]:
labels = ["apple", "orange", "banana"]
labels_map = {k: v for v, k in enumerate(labels)}
with open("prepare_data/obj_det_ssd_json/labels_map.json", "w") as fp:
    json.dump(labels_map, fp)
labels_map

{'apple': 0, 'orange': 1, 'banana': 2}

<IPython.core.display.Javascript object>

In [7]:
gt_df["class_id"] = gt_df["label_name"].map(labels_map)
gt_df.head()

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height,class_id
0,apple,76,31,410,350,7385219.jpg,705,724,0
1,orange,169,172,396,465,7385219.jpg,705,724,1
2,apple,143,54,357,341,3503298.jpg,730,725,0
3,banana,101,186,481,363,3503298.jpg,730,725,2
4,apple,81,40,473,332,3503298.jpg,730,725,0


<IPython.core.display.Javascript object>

In [8]:
columns_map = {
    "bbox_x": "left",
    "bbox_y": "top",
    "bbox_width": "width",
    "bbox_height": "height",
}

gt_df = gt_df.rename(columns=columns_map)
gt_df.head()

,label_name,left,top,width,height,image_name,image_width,image_height,class_id
0,apple,76,31,410,350,7385219.jpg,705,724,0
1,orange,169,172,396,465,7385219.jpg,705,724,1
2,apple,143,54,357,341,3503298.jpg,730,725,0
3,banana,101,186,481,363,3503298.jpg,730,725,2
4,apple,81,40,473,332,3503298.jpg,730,725,0


<IPython.core.display.Javascript object>

In [9]:
categories = []
for k, v in labels_map.items():
    categories.append({"class_id": v, "name": k})
print(categories)

[{'class_id': 0, 'name': 'apple'}, {'class_id': 1, 'name': 'orange'}, {'class_id': 2, 'name': 'banana'}]


<IPython.core.display.Javascript object>

In [10]:
images = gt_df["image_name"].unique().tolist()
for image in images:
    img_df = gt_df[gt_df["image_name"] == image]
    image_size = [
        {
            "width": int(img_df["image_width"].iloc[0]),
            "height": int(img_df["image_height"].iloc[0]),
            "depth": 3,
        }
    ]

    annotations = img_df[["class_id", "left", "top", "width", "height"]].to_dict(
        orient="records"
    )

    gt_dct = {
        "file": image,
        "image_size": image_size,
        "annotation": annotations,
        "categories": categories,
    }

    if random.random() < 0.7:
        shutil.copy(
            "prepare_data/obj_det_ssd_json/all_images_gt/" + image,
            "prepare_data/obj_det_ssd_json/train_imgs/",
        )
        json_name = (
            "prepare_data/obj_det_ssd_json/train_annots/"
            + image.split(".")[0]
            + ".json"
        )
        with open(json_name, "w") as fp:
            json.dump(gt_dct, fp)
    else:
        shutil.copy(
            "prepare_data/obj_det_ssd_json/all_images_gt/" + image,
            "prepare_data/obj_det_ssd_json/valid_imgs/",
        )
        json_name = (
            "prepare_data/obj_det_ssd_json/valid_annots/"
            + image.split(".")[0]
            + ".json"
        )
        with open(json_name, "w") as fp:
            json.dump(gt_dct, fp)

<IPython.core.display.Javascript object>

In [11]:
print(json.dumps(gt_dct, indent=4))

{
    "file": "8278941.jpg",
    "image_size": [
        {
            "width": 676,
            "height": 797,
            "depth": 3
        }
    ],
    "annotation": [
        {
            "class_id": 2,
            "left": 74,
            "top": 121,
            "width": 425,
            "height": 363
        },
        {
            "class_id": 0,
            "left": 59,
            "top": 148,
            "width": 482,
            "height": 352
        }
    ],
    "categories": [
        {
            "class_id": 0,
            "name": "apple"
        },
        {
            "class_id": 1,
            "name": "orange"
        },
        {
            "class_id": 2,
            "name": "banana"
        }
    ]
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>